In [ ]:
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["dadosVivamente"]
collection = db["postsComPerguntasComInfos"]

In [ ]:
# Filtrando documentos com o atributo nivel maior ou igual a 2
filtro = {"nivel": {"$gte": 2}}
documentos = collection.find(filtro)

In [ ]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))

In [ ]:
df.head()

In [ ]:
df['postCreated_time'] = pd.to_datetime(df['postCreated_time'])
df = df.sort_values(by='postCreated_time')

In [ ]:
df['date'] = df['postCreated_time'].dt.date
df_grouped = df.groupby(['id_usuario', 'date']).size().reset_index(name='post_count')

# Escolha um usuário específico para visualizar os dados, ou faça a análise em todos os usuários
usuario_id = df_grouped['id_usuario'].iloc[0]  # aqui pegamos o primeiro usuário como exemplo
df_usuario = df_grouped[df_grouped['id_usuario'] == usuario_id]

# Exibindo a série temporal de contagem de postagens
plt.figure(figsize=(10, 6))
plt.plot(df_usuario['date'], df_usuario['post_count'], label='Post Count')
plt.title(f'Post Count Over Time for User {usuario_id}')
plt.xlabel('Date')
plt.ylabel('Post Count')
plt.xticks(rotation=45)
plt.legend()
plt.show()

# Implementando Média Móvel (Moving Average)
window_size = 7  # por exemplo, uma janela de 7 dias
df_usuario['moving_average'] = df_usuario['post_count'].rolling(window=window_size).mean()

# Plotando Média Móvel
plt.figure(figsize=(10, 6))
plt.plot(df_usuario['date'], df_usuario['post_count'], label='Original')
plt.plot(df_usuario['date'], df_usuario['moving_average'], label=f'{window_size}-Day Moving Average', color='orange')
plt.title(f'Moving Average of Post Count Over Time for User {usuario_id}')
plt.xlabel('Date')
plt.ylabel('Post Count')
plt.xticks(rotation=45)
plt.legend()
plt.show()

# Implementando Suavização Exponencial (Exponential Smoothing)
alpha = 0.3  # taxa de suavização
df_usuario['exponential_smoothing'] = df_usuario['post_count'].ewm(alpha=alpha).mean()

# Plotando Suavização Exponencial
plt.figure(figsize=(10, 6))
plt.plot(df_usuario['date'], df_usuario['post_count'], label='Original')
plt.plot(df_usuario['date'], df_usuario['exponential_smoothing'], label=f'Exponential Smoothing (α={alpha})', color='green')
plt.title(f'Exponential Smoothing of Post Count Over Time for User {usuario_id}')
plt.xlabel('Date')
plt.ylabel('Post Count')
plt.xticks(rotation=45)
plt.legend()
plt.show()